# Evaluate Embedding Model for use in Retrieval

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from scipy.stats import skew

# model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
model = SentenceTransformer("all-mpnet-base-v2")
# model = SentenceTransformer("all-MiniLM-L6-v2")

/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
def embed(model: SentenceTransformer, x: str, method: str = "cosine"):
    y = model.encode(x)
    y = y[np.newaxis, :]
    return y

# Documents

The following information is scraped from Wikipedia articles of various topics. OpenAI's GPT4o has generated short sentences and queries about the articles.

In [3]:
llama_statements = [
    "The llama (/ˈlɑːmə/; Spanish pronunciation: [ˈʎama] or [ˈʝama]) is a domesticated South American camelid.",
    "Llamas have been used by Andean cultures as meat and pack animals since the pre-Columbian era.",
    "Llamas are social animals that live in herds.",
    "Their wool is soft and contains only a small amount of lanolin.",
    "Llamas can learn simple tasks after a few repetitions.",
    "They can carry about 25 to 30% of their body weight for 8 to 13 km (5–8 miles).",
    "The name 'llama' was adopted by European settlers from native Peruvians.",
    "Llamas' ancestors originated from the Great Plains of North America about 40 million years ago.",
    "They migrated to South America about three million years ago.",
    "This migration occurred during the Great American Interchange.",
    "Camelids became extinct in North America by the end of the last ice age (10,000–12,000 years ago).",
    "As of 2007, there were over seven million llamas and alpacas in South America.",
    "The United States and Canada had over 158,000 llamas and 100,000 alpacas in 2007.",
    "These North American populations descended from llamas imported in the late 20th century.",
    "Llamas are used for carrying loads in Andean cultures.",
    "The word 'llama' was sometimes spelled 'lama' or 'glama' in the past.",
    "Llamas are capable of transporting loads over long distances.",
    "Llamas are known for their soft wool.",
    "European settlers learned about llamas from the indigenous people of Peru.",
    "The llama's migration to South America was a significant event in its evolutionary history.",
    "Llamas belong to the species Lama glama.",
    "They are widely used as pack animals in the Andes.",
    "Llamas can live and work in harsh mountainous environments.",
    "The soft wool of llamas is prized for its quality.",
    "Llamas are capable of adapting to various tasks.",
    "They play an important role in Andean agriculture and transportation.",
    "Llamas communicate using a series of vocalizations.",
    "The llama's scientific name is Lama glama.",
    "They have been domesticated for thousands of years.",
    "Llamas are closely related to alpacas, vicuñas, and guanacos.",
    "They have a lifespan of about 15 to 25 years.",
    "Llamas are known for their gentle and calm demeanor.",
    "Their ancestors once roamed North America.",
    "Llamas have a unique split-upper lip.",
    "They can thrive on a diet of grass and hay.",
    "Llamas are often used in therapy and educational programs.",
    "Their wool comes in a variety of natural colors.",
    "Llamas have been depicted in ancient Andean art and artifacts.",
    "They are an integral part of Andean cultural heritage.",
    "Llamas can form strong bonds with humans and other animals.",
    "Llamas can spit as a defense mechanism.",
    "They have padded feet that are well-suited for mountainous terrain.",
    "Llamas are known for their long necks and large eyes.",
    "They can weigh between 130 and 200 kilograms (290–440 pounds).",
    "Llamas are herbivores, primarily eating grass and hay.",
    "They have a three-compartment stomach for efficient digestion.",
    "Llamas are often used in trekking and hiking tours.",
    "They have a strong sense of curiosity.",
    "Llamas can be trained to pull carts.",
    "Their wool is hypoallergenic.",
    "Llamas have a gestation period of about 11.5 months.",
    "A baby llama is called a cria.",
    "Llamas are important to the rural economy in the Andes.",
    "They can survive with minimal water in arid environments.",
    "Llamas have a strong herding instinct.",
    "They can recognize individual humans and animals.",
    "Llamas' wool is often used for making textiles and clothing.",
    "They are sometimes used as guard animals for livestock.",
    "Llamas have been featured in various films and media.",
    "They play a significant role in traditional Andean festivals and rituals.",
]

alpaca_statements = [
    "The alpaca (Lama pacos) is a South American camelid mammal.",
    "Alpacas traditionally graze on the Andes' high plateaus.",
    "They are found in Southern Peru, Western Bolivia, Ecuador, and Northern Chile.",
    "Today, alpacas are raised globally on farms and ranches.",
    "Thousands of alpacas are born annually in North America, Europe, and Australia.",
    "There are two main breeds of alpaca: the Suri and the Huacaya.",
    "Suri alpacas produce straight 'locks' of fiber.",
    "Huacaya alpacas have crimped, wavy wool.",
    "Both types of alpaca fiber are highly valued.",
    "Alpaca fiber is used for knitted and woven items.",
    "Alpacas are similar in appearance to llamas.",
    "Alpacas are shorter than llamas.",
    "Alpacas are primarily bred for their wool.",
    "Llamas are often used as livestock guardians.",
    "Llamas also serve as pack animals.",
    "All four South American camelids are closely related.",
    "Alpacas and llamas can interbreed.",
    "Alpacas are believed to be domesticated from the vicuña.",
    "Llamas are thought to descend from the guanaco.",
    "Domestication of alpacas and llamas occurred 5,000 to 6,000 years ago.",
]

cattle_statements = [
    "Cattle (Bos taurus) are large, domesticated, bovid ungulates widely kept as livestock.",
    "They are prominent modern members of the subfamily Bovinae.",
    "Cattle are the most widespread species of the genus Bos.",
    "Mature female cattle are called cows.",
    "Mature male cattle are called bulls.",
    "Young female cattle are called heifers.",
    "Young male cattle are called oxen or bullocks.",
    "Castrated male cattle are known as steers.",
    "Cattle are commonly raised for meat.",
    "They are also raised for dairy products.",
    "Cattle are raised for leather as well.",
    "As draft animals, cattle pull carts and farm implements.",
    "In India, cattle are sacred animals within Hinduism.",
    "Cattle may not be killed in Hinduism.",
    "Small breeds like the miniature Zebu are kept as pets.",
    "Taurine cattle are widely distributed across Europe and temperate areas of Asia.",
    "Zebus are found mainly in India and tropical areas of Asia, America, and Australia.",
    "Sanga cattle are found primarily in sub-Saharan Africa.",
    "These types of cattle are sometimes classified as separate species or subspecies.",
    "There are over 1,000 recognized breeds of cattle.",
]

plastic_statements = [
    "A TPU resin consists of linear polymeric chains in block-structures.",
    "These chains contain low polarity segments which are rather long, called soft segments.",
    "Alternating with the soft segments are shorter, high polarity segments known as hard segments.",
    "Both types of segments in TPU are linked by covalent links.",
    "The segments form block-copolymers in TPU.",
    "The miscibility of the hard and soft segments depends on their glass transition temperature (Tg).",
    "Tg occurs at the onset of micro-Brownian segmental motion.",
    "It can be identified by dynamic mechanical spectra.",
    "For an immiscible TPU, the loss modulus spectrum typically shows double peaks.",
    "Each peak is assigned to the Tg of one component.",
    "If the two components are miscible, TPU shows a single broad peak.",
    "This peak's position lies between that of the two original Tg peaks.",
    "The polarity of the hard segments creates a strong attraction between them.",
    "This attraction causes a high degree of aggregation and order in this phase.",
    "Crystalline or pseudo-crystalline areas form in a soft and flexible matrix.",
    "Phase separation between blocks can vary based on polarity and molecular weight.",
    "The crystalline areas act as physical cross-links, giving TPU its high elasticity.",
    "The flexible chains provide elongation characteristics to the polymer.",
    "Pseudo cross-links disappear under heat, allowing for classical processing methods.",
    "TPU scrap can be reprocessed due to the thermal properties of the material.",
    "TPU plastic is known for its excellent elasticity.",
    "It has superior abrasion resistance.",
    "TPU is widely used in various industrial applications.",
    "This plastic offers high tensile strength.",
    "TPU can withstand low temperatures without becoming brittle.",
    "It is resistant to oil and grease.",
    "TPU plastic is often used in the production of footwear.",
    "It has a high level of transparency.",
    "TPU can be easily colored with different dyes.",
    "It is commonly used in the manufacturing of medical devices.",
    "TPU plastic is flexible and durable.",
    "It is often used to make phone cases.",
    "TPU is resistant to many chemicals.",
    "It is used in automotive parts due to its strength.",
    "TPU can be processed by injection molding.",
    "It is available in various hardness levels.",
    "TPU plastic is known for its versatility.",
    "It is used in the production of sporting goods.",
    "TPU is environmentally friendly and recyclable.",
    "This plastic has excellent impact resistance.",
]

# Questions

In [4]:
llama_questions = [
    "What is the scientific name for a llama?",
    "Where are llamas originally from?",
    "How long is the typical lifespan of a llama?",
    "What do llamas primarily eat?",
    "How much does a fully grown llama typically weigh?",
    "What is the difference between a llama and an alpaca?",
    "How many stomach compartments do llamas have?",
    "What is the average height of a llama at the shoulder?",
    "How do llamas communicate with each other?",
    "What are the main uses of llamas by humans?",
    "How do llamas react when they feel threatened?",
    "What type of climate is best suited for llamas?",
    "What is the gestation period for a llama?",
    "How often do llamas need to be sheared?",
    "What is a group of llamas called?",
    "How can you tell the age of a llama?",
    "What are some common health issues that llamas face?",
    "How do llamas contribute to their ecosystems?",
    "Can llamas be used as guard animals?",
    "What are the main differences in behavior between wild and domesticated llamas?",
]

alpaca_questions = [
    "What is the average lifespan of an alpaca?",
    "What are the two types of alpacas, and how do they differ?",
    "What is the primary use of alpacas in farming?",
    "Where do alpacas originate from?",
    "What is the typical diet of an alpaca?",
    "How often should alpacas be sheared?",
    "What are some common health issues that alpacas face?",
    "How do alpacas communicate with each other?",
    "What is the gestation period for an alpaca?",
    "How can you distinguish between a male and a female alpaca?",
    "What are the different colors of alpaca fleece?",
    "How does the quality of alpaca fleece compare to sheep's wool?",
    "What are some products made from alpaca fleece?",
    "How much fleece does an alpaca produce annually?",
    "What is the average weight of an adult alpaca?",
    "How do alpacas behave in social settings with other alpacas?",
    "What are some signs of a healthy alpaca?",
    "How can you tell if an alpaca is stressed or unhappy?",
    "What kind of shelter do alpacas need?",
    "How do alpacas contribute to the ecosystem?",
]

cattle_questions = [
    "What are the main differences between beef cattle and dairy cattle?",
    "How long is the gestation period for a cow?",
    "What is the average lifespan of a domesticated cow?",
    "What are the primary breeds of beef cattle?",
    "How much milk does a dairy cow produce on average per day?",
    "What is the purpose of dehorning cattle?",
    "How do farmers typically identify and track their cattle?",
    "What are the common health issues faced by cattle?",
    "How is IVF used in cattle breeding?",
    "What are the nutritional requirements for a growing calf?",
    "What is the significance of the rumen in a cow's digestive system?",
    "How does climate change impact cattle farming?",
    "What are the benefits of rotational grazing for cattle?",
    "How do cattle contribute to greenhouse gas emissions?",
    "What are the key indicators of a cow's health and well-being?",
    "How has the domestication of cattle influenced human agriculture?",
    "What are the ethical considerations in cattle farming?",
    "How do cattle farmers ensure the welfare of their animals during transport?",
    "What are the common methods used for weaning calves from their mothers?",
    "How is biotechnology used to improve cattle breeds and productivity?",
]

plastic_questions = [
    "What does TPU stand for in the context of plastics?",
    "What are the primary uses of TPU plastic in manufacturing?",
    "How does the flexibility of TPU plastic compare to other plastics?",
    "What are the main advantages of using TPU plastic over traditional rubber?",
    "Can TPU plastic be recycled, and if so, how?",
    "What are the key chemical properties of TPU plastic?",
    "How is TPU plastic produced?",
    "What industries commonly use TPU plastic for their products?",
    "How does TPU plastic perform in high-temperature environments?",
    "What are the environmental impacts of producing TPU plastic?",
    "Is TPU plastic considered biodegradable?",
    "How does the hardness of TPU plastic vary with different formulations?",
    "What are some common consumer products made from TPU plastic?",
    "How does the transparency of TPU plastic compare to other thermoplastics?",
    "What are the tensile strength and elongation properties of TPU plastic?",
    "How does TPU plastic react to exposure to UV radiation?",
    "What are the differences between TPU plastic and TPE plastic?",
    "How does TPU plastic handle exposure to chemicals and oils?",
    "What are some limitations or disadvantages of TPU plastic?",
    "How does the cost of TPU plastic compare to other types of plastics?",
]

In [5]:
document_embeddings = [
    [
        embed(model, statement)
        for statement in topic
    ]
    for topic in [
        llama_statements,
        alpaca_statements,
        cattle_statements,
        plastic_statements,
    ]
]
query_embeddings = [
    [
        embed(model, question)
        for question in topic
    ]
    for topic in [
        llama_questions,
        alpaca_questions,
        cattle_questions,
        plastic_questions,
    ]
]
labels = ["llama", "alpaca", "cattle", "plastic"]

# Compute Metrics

In [6]:
from embedding_metrics import compute_cvm, compute_ks, create_dataframe

### Document-to-Document

In [7]:
# document-to-document
cvm_doc = compute_cvm(document_embeddings, labels)
ks_doc = compute_ks(document_embeddings, labels)

# create dataframes
cvm_doc_stats, cvm_doc_pvals = create_dataframe(cvm_doc, labels)
ks_doc_stats, ks_doc_pvals = create_dataframe(ks_doc, labels)

Cramer-von Mises

In [8]:
cvm_doc_stats

Reference                           
                  llama   alpaca   cattle  plastic
Query llama       2.161    5.797   80.846  424.460
      alpaca     15.184    0.447   46.029   83.134
      cattle     38.783   21.611    1.117   79.674
      plastic   242.351  106.144  140.168    0.063

In [9]:
cvm_doc_pvals

Reference                      
                  llama alpaca cattle plastic
Query llama         0.0  0.000  0.000   0.000
      alpaca        0.0  0.051  0.000   0.000
      cattle        0.0  0.000  0.001   0.000
      plastic       0.0  0.000  0.000   0.809

Kolmgorov-Smirnov

In [10]:
ks_doc_stats

Reference                      
                  llama alpaca cattle plastic
Query llama       0.441  0.153  0.460   0.864
      alpaca      0.356  0.335  0.728   0.921
      cattle      0.621  0.559  0.562   0.876
      plastic     0.762  0.649  0.790   0.135

In [11]:
ks_doc_pvals

Reference                      
                  llama alpaca cattle plastic
Query llama         0.0   0.00  0.000   0.000
      alpaca        0.0   0.18  0.000   0.000
      cattle        0.0   0.00  0.002   0.000
      plastic       0.0   0.00  0.000   0.819

### Query-to-Document

In [12]:
# query-to-document
cvm_query_doc = compute_cvm(
    queries=query_embeddings,
    references=document_embeddings,
    classes=labels,
)
ks_query_doc = compute_ks(
    queries=query_embeddings,
    references=document_embeddings,
    classes=labels,
)

# create dataframes
cvm_query_doc_stats, cvm_query_doc_pvals = create_dataframe(cvm_query_doc, labels)
ks_query_doc_stats, ks_query_doc_pvals = create_dataframe(ks_query_doc, labels)

In [13]:
cvm_query_doc_stats

Reference                        
                  llama  alpaca  cattle plastic
Query llama       0.163  45.693  63.451  64.958
      alpaca     28.348   0.164  59.755  64.847
      cattle     44.782  44.724   0.163  64.829
      plastic    64.958  64.918  64.958   0.164

In [14]:
cvm_query_doc_pvals

Reference                      
                  llama alpaca cattle plastic
Query llama       0.352  0.000  0.000   0.000
      alpaca      0.000  0.351  0.000   0.000
      cattle      0.000  0.000  0.352   0.000
      plastic     0.000  0.000  0.000   0.351

In [15]:
ks_query_doc_stats

Reference                      
                  llama alpaca cattle plastic
Query llama       0.050  0.762  0.983   1.000
      alpaca      0.581  0.050  0.894   0.998
      cattle      0.694  0.726  0.050   0.998
      plastic     1.000  0.998  1.000   0.050

In [16]:
ks_query_doc_pvals

Reference                      
                  llama alpaca cattle plastic
Query llama       0.689  0.000  0.000   0.000
      alpaca      0.000  0.689  0.000   0.000
      cattle      0.000  0.000  0.689   0.000
      plastic     0.000  0.000  0.000   0.689